In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
import plotly.express as px

In [115]:
df = pd.read_csv('/Users/nishant/Desktop/Datasets/Phonepe/aggregated/Phonepe/phonepe_transaction.csv')

In [116]:
# Convert all column names to lowercase for consistency
df.columns = df.columns.str.lower()
print("Column names converted to lowercase:")
print(df.columns)

Column names converted to lowercase:
Index(['location_type', 'country', 'state', 'location_name', 'quarter',
       'from_date', 'to_date', 'transaction_category', 'payment_type',
       'transaction_count', 'transaction_amount', 'transaction_amount_crores',
       'avg_transaction_value'],
      dtype='object')


In [117]:
print(df.columns)

Index(['location_type', 'country', 'state', 'location_name', 'quarter',
       'from_date', 'to_date', 'transaction_category', 'payment_type',
       'transaction_count', 'transaction_amount', 'transaction_amount_crores',
       'avg_transaction_value'],
      dtype='object')


In [118]:
df.head()

,location_type,country,state,location_name,quarter,from_date,to_date,transaction_category,payment_type,transaction_count,transaction_amount,transaction_amount_crores,avg_transaction_value
0,national,India,All States,National,Q1 2018,01/01/18,28/03/18,Recharge & bill payments,TOTAL,72550406,1.447271e+10,1447.271356,199.484942
1,national,India,All States,National,Q1 2018,01/01/18,28/03/18,Peer-to-peer payments,TOTAL,46982705,1.472460e+11,14724.588350,3134.044401
2,national,India,All States,National,Q1 2018,01/01/18,28/03/18,Merchant payments,TOTAL,5368669,4.656679e+09,465.667891,867.380521
3,national,India,All States,National,Q1 2018,01/01/18,28/03/18,Financial Services,TOTAL,3762820,8.158531e+08,81.585311,216.819594
4,national,India,All States,National,Q1 2018,01/01/18,28/03/18,Others,TOTAL,5761576,4.643217e+09,464.321730,805.893613


In [119]:
df.shape

(5174, 13)

In [120]:
df.describe()

,transaction_count,transaction_amount,transaction_amount_crores,avg_transaction_value
count,5.174000e+03,5.174000e+03,5.174000e+03,5174.000000
mean,9.094881e+07,1.335613e+11,1.335613e+04,1358.392983
std,6.254713e+08,1.002325e+12,1.002325e+05,1335.776079
min,2.000000e+00,3.439721e+01,3.440000e-06,17.198606
25%,6.363575e+04,4.423959e+07,4.423959e+00,511.401038
50%,5.737635e+05,4.846731e+08,4.846731e+01,797.456752
75%,1.361543e+07,1.255325e+10,1.255325e+03,1516.534455
max,1.741919e+10,2.663390e+13,2.663387e+06,7767.539936


In [121]:
df.isnull().sum() # Check for missing values

location_type                0
country                      0
state                        0
location_name                0
quarter                      0
from_date                    0
to_date                      0
transaction_category         0
payment_type                 0
transaction_count            0
transaction_amount           0
transaction_amount_crores    0
avg_transaction_value        0
dtype: int64

Top 10 States by Transaction Amount and Count:

In [122]:
# Filter out 'All States'
df_filtered = df[df['state'] != 'All States']
top_10_states_amount = df_filtered.groupby('state')['transaction_amount'].sum().sort_values(ascending=False).head(10)
# Convert the transaction amount to crores
top_10_states_amount_crores = (top_10_states_amount / 1e7).round(2)
print("Top 10 States by Transaction Amount (in Crores):")
print(top_10_states_amount_crores)


Top 10 States by Transaction Amount (in Crores):
state
Telangana         4165593.80
Karnataka         4067871.88
Maharashtra       4037418.51
Andhra Pradesh    3466908.86
Uttar Pradesh     2688522.23
Rajasthan         2634323.70
Madhya Pradesh    1912527.11
Bihar             1790135.09
West Bengal       1558415.94
Odisha            1226398.27
Name: transaction_amount, dtype: float64


In [123]:
# 2. Visualize the top 10 states by transaction amount
fig_amount = px.bar(top_10_states_amount_crores,
                    x=top_10_states_amount_crores.index,
                    y='transaction_amount',
                    title='Top 10 States by Transaction Amount',
                    labels={'transaction_amount': 'Total Transaction Amount (in Crores)', 'x': 'State'})
fig_amount.show()

In [124]:
# Group by quarter and sum the transaction amount
quarterly_amount = df_filtered.groupby('quarter')['transaction_amount'].sum().reset_index()

# Create a bar chart to visualize the quarterly transaction amount
fig_quarterly_amount = px.bar(quarterly_amount,
                            x='quarter',
                            y='transaction_amount',
                            title='Transaction Amount by Quarter',
                            labels={'transaction_amount': 'Total Transaction Amount', 'quarter': 'Quarter'},
                            color='quarter')
fig_quarterly_amount.show()

In [125]:
# Group by quarter and sum the transaction count
quarterly_count = df_filtered.groupby('quarter')['transaction_count'].sum().reset_index()

# Create a bar chart to visualize the quarterly transaction count
fig_quarterly_count = px.bar(quarterly_count,
                           x='quarter',
                           y='transaction_count',
                           title='Transaction Count by Quarter',
                           labels={'transaction_count': 'Total Transaction Count', 'quarter': 'Quarter'},
                           color='quarter')
fig_quarterly_count.show()

4. Most Common Transaction Types


In [126]:
# Group by transaction category and sum the transaction count
transaction_category_counts = df_filtered.groupby('transaction_category')['transaction_count'].sum().reset_index()

# Create a pie chart to visualize the distribution of transaction categories
fig_pie_categories = px.pie(transaction_category_counts,
                       names='transaction_category',
                       values='transaction_count',
                       title='Distribution of Transaction Categories',
                       hole=0.05)
fig_pie_categories.update_traces(textposition='inside', textinfo='percent+label')
fig_pie_categories.show()                       

5. Distribution of Transaction Types in a Specific State


In [127]:
# Choose a state to analyze (with correct capitalization)
state_to_analyze = 'Bihar'

# Filter the DataFrame for the selected state
state_df = df_filtered[df_filtered['state'] == state_to_analyze]

# Group by the correct column name: 'transaction_category'
state_transaction_counts = state_df.groupby('transaction_category')['transaction_count'].sum().reset_index()

# Create a pie chart
fig_state_pie = px.pie(state_transaction_counts,
                       names='transaction_category',
                       values='transaction_count',
                       title=f'Distribution of Transaction Types in {state_to_analyze}',
                       hole=0.05)

fig_state_pie.update_traces(textposition='inside', textinfo='percent+label')
fig_state_pie.show()

6. Transaction Amount vs. Transaction Count by State

In [128]:
# Group by state to get the total transaction amount and count
state_summary = df_filtered.groupby('state').agg({
    'transaction_amount':'sum',
    'transaction_count':'sum'
}).reset_index()

In [129]:
# create a bar chart to visualize the relationship
fig_bar_comparison = px.bar(state_summary,
                            x='state',
                            y=['transaction_amount', 'transaction_count'],
                            title='Transaction Amount vs. Transaction Count by State',
                            labels={'value': 'Transaction Amount / Count', 'variable': 'Metric'},
                            barmode='group',# This creates the side-by-side bars
                            log_y=True)  # Use log scale for better visibility
fig_bar_comparison.update_layout(xaxis={'categoryorder':'total descending'}) # Order states by the total value
fig_bar_comparison.show()

7. Average Transaction Value (ATV) by State

In [130]:
state_summary['average_transaction_value'] = state_summary['transaction_amount'] / state_summary['transaction_count']

# sort the state by ATV in desc order
atv_sorted = state_summary.sort_values(by='average_transaction_value',ascending=False)


In [131]:
# Get the top 10 and bottom 10 states by ATV
top_10_atv = atv_sorted.head(10)
bottom_10_atv = atv_sorted.tail(10)

# Combine them into a single dataframe for plotting
atv_comparison = pd.concat([top_10_atv, bottom_10_atv])

# Create the bar chart
fig_atv_bar = px.bar(atv_comparison,
                     x='state',
                     y='average_transaction_value',
                     color='average_transaction_value',
                     color_continuous_scale='Plasma',
                     title='Top and Bottom 10 States by Average Transaction Value (ATV)',
                     labels={'average_transaction_value': 'Average Transaction Value', 'state': 'State'})

fig_atv_bar.update_layout(xaxis={'categoryorder':'total descending'})
fig_atv_bar.show()

Correlation Analysis of Key Metrics

In [132]:
# --- Calculate the Correlation Matrix ---

# Select only the numerical columns for correlation analysis
correlation_data = state_summary[['transaction_amount', 'transaction_count', 'average_transaction_value']]

# Calculate the correlation matrix
correlation_matrix = correlation_data.corr()

# Print the correlation matrix for reference
print("Correlation Matrix:")
print(correlation_matrix)


# --- Visualize the Correlation Matrix with a Heatmap ---

# Create the heatmap
fig_corr_heatmap = px.imshow(
    correlation_matrix,
    text_auto=True,  # Automatically display the correlation values on the heatmap
    aspect="auto",
    color_continuous_scale='RdBu_r', # Use a diverging color scale for better interpretation
    title='Correlation Matrix of Transaction Metrics'
)

fig_corr_heatmap.show()

Correlation Matrix:
                           transaction_amount  transaction_count  \
transaction_amount                   1.000000           0.988798   
transaction_count                    0.988798           1.000000   
average_transaction_value           -0.255306          -0.293606   

                           average_transaction_value  
transaction_amount                         -0.255306  
transaction_count                          -0.293606  
average_transaction_value                   1.000000  
